TEXT

In [91]:
# IMPORTS
import random
import numpy as np
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, HillClimbSearch, BicScore
from pgmpy.sampling import BayesianModelSampling

In [8]:
# Set random seed to ensure reproducibility
random.seed(37)

In [3]:
file_path = 'instances_01_KP/low-dimensional/f1_l-d_kp_10_269'
data = np.loadtxt(file_path, dtype=int)
col_1 = data[:, 0]
col_2 = data[:, 1]

n_items = col_1[0]
capacity = col_2[0]

values = data[1:, 0]
weights = data[1:, 1]

# print("First Column:", col_1)
# print("Second Column:", col_2)
print("number of items:", n_items)
print("max weight:", capacity)
print("values:", values)
print("weights:", weights)

number of items: 10
max weight: 269
values: [55 10 47  5  4 50  8 61 85 87]
weights: [95  4 60 32 23 72 80 62 65 46]


In [4]:
def knapsack_fitness(solution, values, weights, capacity):
    total_weight = np.dot(solution, weights)
    total_value = np.dot(solution, values)
    if total_weight > capacity:
        return 0  # Invalid solution
    return total_value

In [5]:
def generate_random_solution(length):
    return np.random.randint(2, size=length)

In [6]:
test_sol = generate_random_solution(n_items)
test_sol_fitness = knapsack_fitness(test_sol, values, weights, capacity)
print(test_sol)
print(test_sol_fitness)

[0 1 0 0 1 1 0 0 1 0]
149


In [66]:
# Store problem items in dictionary

items = {}
for i in range(n_items):
    items[i] = (values[i], weights[i])

print(items)
print(len(items))
print(items[3][1])

{0: (55, 95), 1: (10, 4), 2: (47, 60), 3: (5, 32), 4: (4, 23), 5: (50, 72), 6: (8, 80), 7: (61, 62), 8: (85, 65), 9: (87, 46)}
10
32


In [98]:
# Initialise fitness
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# weights is positive to signify maximisation of fitness, use negative to minimise
# weights must be a tuple to allow to single and multi objective problems
# to be treated in the same way

# Initialise individual
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = len(items) # number of genes in individual

def evalIndividual(individual):
    weight = 0
    value = 0
    # print(individual)
    for i in range(IND_SIZE):
        value += items[i][0] * individual[i]
        weight += items[i][1] * individual[i]
    if weight > capacity:
        return (0,)
    print(value)
    return (value,)

toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalIndividual)
toolbox.register("select", tools.selBest)

def fit_bayesian_network(population):
    # convert population to data for model
    data = np.array(population)

    # learn structure of bayesian network
    hc = HillClimbSearch(pd.DataFrame(data))

    # determine structure of model
    model_structure = hc.estimate(scoring_method=BicScore(pd.DataFrame(data)))

    print(f"Data shape: {pd.DataFrame(data).shape}")
    print(f"Data columns: {pd.DataFrame(data).columns}")

    missing_nodes = set(range(10)) - set(model_structure.nodes())
    for node in missing_nodes:
        model_structure.add_node(node)

    # determine structure of model
    # model_structure = hc.estimate(scoring_method=BicScore(pd.DataFrame(data)))

    # create model using best structure
    model = BayesianNetwork(model_structure.edges())

    # learn parameters and return model
    model.fit(pd.DataFrame(data), estimator=MaximumLikelihoodEstimator)
    print(model.nodes())  # Should list all 10 variables (nodes)
    print(model.edges())  # Should show the relationships between the variables
    return model

def sample_population_from_bayesian(bayesian_network, pop_size):
    # generate individuals by sampling bayesian netwrok
    inference = BayesianModelSampling(bayesian_network)
    new_population = inference.forward_sample(size=pop_size).values.tolist()
    return new_population

def BOA(pop_size, n_generations, n_selected):
    gen = 0
    # initialise population
    population = toolbox.population(pop_size)
    print(population)

    # loop through each generation
    for generation in range(n_generations):
        fitnesses = list(map(toolbox.evaluate, population)) # eval pop

        for ind, fit in zip(population, fitnesses):
            print(fit)
            ind.fitness.values = fit
        
        # print best in population
        top_individual = tools.selBest(population, 1)[0]
        print(f"Generation {gen}: Best Individual = {top_individual}, Fitness = {top_individual.fitness.values}")
        gen += 1

        # select best individuals from population
        selected_population = toolbox.select(population, n_selected)

        # learn bayesian network from selected individuals
        network = fit_bayesian_network(selected_population)

        # generate new population by sampling bayesian network
        new_population = sample_population_from_bayesian(network, len(population))

        # replace population
        population[:] = [creator.Individual(ind) for ind in new_population]


In [100]:
BOA(10, 10, 7)

[[0, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 0, 1, 1, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 1, 1, 0], [0, 1, 1, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0, 1, 0, 1], [1, 0, 0, 0, 1, 1, 1, 0, 1, 0], [0, 1, 0, 1, 0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, 1, 1, 1, 1]]
140
193
147
157
169
4
(0,)
(0,)
(140,)
(193,)
(147,)
(157,)
(0,)
(169,)
(4,)
(0,)
Generation 0: Best Individual = [0, 0, 1, 0, 0, 0, 0, 1, 1, 0], Fitness = (193.0,)


  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 9/1000000 [00:00<26:26:06, 10.51it/s] 


Data shape: (7, 10)
Data columns: RangeIndex(start=0, stop=10, step=1)
[2, 8, 3, 1, 4, 9, 6, 5, 7]
[(2, 8), (8, 4), (8, 1), (3, 1), (4, 9), (4, 6), (9, 3), (9, 7), (5, 4)]


Generating for node: 1: 100%|██████████| 9/9 [00:00<00:00, 230.91it/s]


IndexError: list index out of range